In [1]:
import sys
sys.path.append('/home1/04287/mxhf/HETDEX_API/')

In [2]:
%matplotlib inline
import sys
import os
import os.path
import subprocess
import numpy as np

import tables as tb
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.visualization import ZScaleInterval
from astropy.table import Table

from hetdex_api.survey import Survey

In [3]:
survey = Survey('hdr1')

In [8]:
s = Table(survey.hdfile.root.Survey[:])

In [9]:
s = Table([s["datevobs"]])

In [10]:
N = len(s)

In [13]:
s.write("/work/04287/mxhf/maverick/hetdex/shotlist_hdr1.txt", format="ascii")

In [17]:
s[-1]

datevobs
bytes12
20190208v034


In [13]:
from IPython.parallel import Client
c = Client()
lv = c.load_balanced_view()   # this object represents the engines (workers)
print("Using IPython.parallel.")

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-client.json
Using IPython.parallel.


In [ ]:
#build noise model
def run(shotid, DRYRUN=True):
    import os
    import tables as tb
    from astropy.table import Table
    oufilename = "/work/04287/mxhf/maverick/hetdex/cubes{shotid}.images.txt".format(shotid=shotid)
    if os.path.exists(oufilename):
        print("{} exists already.".format(oufilename))
        return 0
    
    print("Processing shot {}.".format(shotid))
    fileh = tb.open_file('/work/03946/hetdex/hdr1/reduction/data/{shotid}.h5'.format(shotid=shotid))
    l = Table( fileh.root.Data.Images[:] )
    l2 = Table( [l["contid"], l["ifuid"], l["ifuslot"], l["specid"]] )
    l2.write(oufilename, format="ascii")
    
    return 0

tasks = []

for shotid in s["datevobs"]:
    tasks.append(lv.apply(run, shotid))

result = [task.get() for task in tasks]  # blocks until all results are back

for r in result:
    print(r)

In [6]:
# non-parallel impleentation
for i, shotid in enumerate(s["datevobs"]):
    oufilename = "/work/04287/mxhf/maverick/hetdex/cubes{shotid}.images.txt".format(shotid=shotid)
    if os.path.exists(oufilename):
        print("{} exists already.".format(oufilename))
        continue
    
    print("Processing shot {} which is {} out of {}.".format(shotid,i,N))
    fileh = tb.open_file('/work/03946/hetdex/hdr1/reduction/data/{shotid}.h5'.format(shotid=shotid))
    l = Table( fileh.root.Data.Images[:] )
    #l.write("{shotid}.images.txt".format(shotid=shotid), format="")
    l2 = Table( [l["contid"], l["ifuid"], l["ifuslot"], l["specid"]] )
    l2.write(oufilename, format="ascii")
    #break

/work/04287/mxhf/maverick/hetdex/cubes20170103v002.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170103v003.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170105v008.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170105v009.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170105v010.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170105v011.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170106v001.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170106v002.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170106v003.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170107v012.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170107v013.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170117v004.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170119v003.images.txt ex

/work/04287/mxhf/maverick/hetdex/cubes20170530v006.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170615v009.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170617v008.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170617v009.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170621v007.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170621v008.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170625v005.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170625v006.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170626v003.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170627v005.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170628v005.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170628v006.images.txt exists already.
/work/04287/mxhf/maverick/hetdex/cubes20170629v004.images.txt ex

Processing shot 20171023v013 which is 262 out of 1714.
Processing shot 20171023v014 which is 263 out of 1714.
Processing shot 20171023v015 which is 264 out of 1714.
Processing shot 20171023v016 which is 265 out of 1714.
Processing shot 20171023v017 which is 266 out of 1714.
Processing shot 20171023v018 which is 267 out of 1714.
Processing shot 20171023v019 which is 268 out of 1714.
Processing shot 20171023v020 which is 269 out of 1714.
Processing shot 20171023v021 which is 270 out of 1714.
Processing shot 20171024v007 which is 271 out of 1714.
Processing shot 20171024v008 which is 272 out of 1714.
Processing shot 20171024v009 which is 273 out of 1714.
Processing shot 20171024v010 which is 274 out of 1714.
Processing shot 20171025v007 which is 275 out of 1714.
Processing shot 20171025v008 which is 276 out of 1714.


KeyboardInterrupt: 

In [18]:
!python create_master_table.py 20190208v034

Processing shot 20190208v034 .
Closing remaining open files:/work/03946/hetdex/hdr1/reduction/data/20190208v034.h5...done


In [20]:
!python ~/hetdex/hetdex-deep/create_master_table.py 20190118v010

Processing shot 20190118v010 .
Closing remaining open files:/work/03946/hetdex/hdr1/reduction/data/20190118v010.h5...done
